# Multi-output neural networks

In [4]:
import numpy as np
import keras
from keras.layers import Dense, Input
from keras.models import Model
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from keras.callbacks import ReduceLROnPlateau
from sklearn.model_selection import GridSearchCV
from scikeras.wrappers import KerasRegressor
from keras.optimizers import Adam


### 1.  
General loss: 15.5   
Main problems:
- ATS and HD difference in buildings
- Regression tasks could be improved
Remarks:
- Categorical cross-entropy required one-hot encoded y values


In [2]:
X_train = pd.read_csv('X_train_C.csv', low_memory=False)
X_test = pd.read_csv('X_test_C.csv', low_memory=False)

Y_train = pd.read_csv('Y_train_C.csv', low_memory=False)
Y_test = pd.read_csv('Y_test_C.csv', low_memory=False)

In [38]:
input_shape = X_train.shape[1] # Number of features

# Define the input layer
input_layer = Input(shape=(input_shape,))

label_encoder = LabelEncoder()

Y_train['Nail spacing [cm]'] = label_encoder.fit_transform(Y_train['Nail spacing [cm]'])
Y_test['Nail spacing [cm]'] = label_encoder.transform(Y_test['Nail spacing [cm]'])

# Use to_categorical
Y_train_onehot = to_categorical(Y_train['Nail spacing [cm]'], num_classes=3)
Y_test_onehot = to_categorical(Y_test['Nail spacing [cm]'], num_classes=3)


# Define separate branches for each output
nail_spacing_output = Dense(3, activation='softmax', name='nail_spacing')(input_layer)
num_sheathing_panels_output = Dense(3, activation='softmax', name='num_sheathing_panels')(input_layer)
num_end_studs_output = Dense(6, activation='softmax', name='num_end_studs')(input_layer)
total_num_studs_output = Dense(1, activation='linear', name='total_num_studs')(input_layer)
holddown_model_output = Dense(1, activation='linear', name='holddown_model')(input_layer)
tx_output = Dense(1, activation='linear', name='tx')(input_layer)
ty_output = Dense(1, activation='linear', name='ty')(input_layer)

# Combine the outputs into a single model
model = Model(inputs=input_layer, outputs=[nail_spacing_output, num_sheathing_panels_output, 
                                                   num_end_studs_output, total_num_studs_output, 
                                                   holddown_model_output, tx_output, ty_output])

# Compile the model
model.compile(optimizer='adam', 
              loss={'nail_spacing': 'categorical_crossentropy',
                    'num_sheathing_panels': 'categorical_crossentropy',
                    'num_end_studs': 'categorical_crossentropy',
                    'total_num_studs': 'mean_squared_error',
                    'holddown_model': 'mean_squared_error',
                    'tx': 'mean_squared_error',
                    'ty': 'mean_squared_error'},
              metrics={'nail_spacing': 'accuracy',
                       'num_sheathing_panels': 'accuracy',
                       'num_end_studs': 'accuracy',
                       'total_num_studs': 'mae',  # Mean Absolute Error for regression tasks
                       'holddown_model': 'mae',
                       'tx': 'mae',
                       'ty': 'mae'})

# Train the model
model.fit(X_train, {'nail_spacing': Y_train_onehot['Nail spacing [cm]'], 
                    'num_sheathing_panels': Y_train['Number sheathing panels'],
                    'num_end_studs': Y_train['Number end studs'],
                    'total_num_studs': Y_train['Total number studs'],
                    'holddown_model': Y_train['HoldDown Model'],
                    'tx': Y_train['Tx(s)'],
                    'ty': Y_train['Ty(s)']},
          epochs=100, batch_size=32, validation_split=0.2)


# Evaluate the model on the test set
model.evaluate(X_test, {'nail_spacing': Y_test['nail_spacing'], 
                        'num_sheathing_panels': Y_test['num_sheathing_panels'],
                        'num_end_studs': Y_test['num_end_studs'],
                        'total_num_studs': Y_test['total_num_studs'],
                        'holddown_model': Y_test['holddown_model'],
                        'tx': Y_test['tx'],
                        'ty': Y_test['ty']})


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [5]:
def label_encoder(Y_train, Y_test):
    label_encoder_nail_spacing = LabelEncoder()
    Y_train['Nail spacing [cm]'] = label_encoder_nail_spacing.fit_transform(Y_train['Nail spacing [cm]'])
    Y_test['Nail spacing [cm]'] = label_encoder_nail_spacing.transform(Y_test['Nail spacing [cm]'])
    Y_train_onehot_nail_spacing = to_categorical(Y_train['Nail spacing [cm]'], num_classes=3)
    Y_test_onehot_nail_spacing = to_categorical(Y_test['Nail spacing [cm]'], num_classes=3)

    label_encoder_num_sheathing_panels = LabelEncoder()
    Y_train['Number sheathing panels'] = label_encoder_num_sheathing_panels.fit_transform(Y_train['Number sheathing panels'])
    Y_test['Number sheathing panels'] = label_encoder_num_sheathing_panels.transform(Y_test['Number sheathing panels'])
    Y_train_onehot_num_sheathing_panels = to_categorical(Y_train['Number sheathing panels'], num_classes=2)
    Y_test_onehot_num_sheathing_panels = to_categorical(Y_test['Number sheathing panels'], num_classes=2)

    label_encoder_num_end_studs = LabelEncoder()
    Y_train['Number end studs'] = label_encoder_num_end_studs.fit_transform(Y_train['Number end studs'])
    Y_test['Number end studs'] = label_encoder_num_end_studs.transform(Y_test['Number end studs'])
    Y_train_onehot_num_end_studs = to_categorical(Y_train['Number end studs'], num_classes=6)
    Y_test_onehot_num_end_studs = to_categorical(Y_test['Number end studs'], num_classes=6)

    return Y_train_onehot_nail_spacing, Y_test_onehot_nail_spacing, Y_train_onehot_num_sheathing_panels, Y_test_onehot_num_sheathing_panels, Y_train_onehot_num_end_studs, Y_test_onehot_num_end_studs, Y_train, Y_test

In [40]:
input_layer = Input(shape=(input_shape,))

# One hot encoding Y
Y_train_onehot_nail_spacing, Y_test_onehot_nail_spacing, Y_train_onehot_num_sheathing_panels, Y_test_onehot_num_sheathing_panels, Y_train_onehot_num_end_studs, Y_test_onehot_num_end_studs, Y_train, Y_test = label_encoder(Y_train, Y_test)

# Define separate branches for each output
nail_spacing_output = Dense(3, activation='softmax', name='nail_spacing')(input_layer)
num_sheathing_panels_output = Dense(2, activation='softmax', name='num_sheathing_panels')(input_layer)
num_end_studs_output = Dense(6, activation='softmax', name='num_end_studs')(input_layer)
total_num_studs_output = Dense(1, activation='linear', name='total_num_studs')(input_layer)
holddown_model_output = Dense(1, activation='linear', name='holddown_model')(input_layer)
tx_output = Dense(1, activation='linear', name='tx')(input_layer)
ty_output = Dense(1, activation='linear', name='ty')(input_layer)

# Combine the outputs into a single model
model = Model(inputs=input_layer, outputs=[nail_spacing_output, num_sheathing_panels_output, 
                                           num_end_studs_output, total_num_studs_output, 
                                           holddown_model_output, tx_output, ty_output])

# Compile the model
model.compile(optimizer='adam', 
              loss={'nail_spacing': 'categorical_crossentropy',
                    'num_sheathing_panels': 'categorical_crossentropy',
                    'num_end_studs': 'categorical_crossentropy',
                    'total_num_studs': 'mean_squared_error',
                    'holddown_model': 'mean_squared_error',
                    'tx': 'mean_squared_error',
                    'ty': 'mean_squared_error'},
              metrics={'nail_spacing': 'accuracy',
                       'num_sheathing_panels': 'accuracy',
                       'num_end_studs': 'accuracy',
                       'total_num_studs': 'mae',
                       'holddown_model': 'mae',
                       'tx': 'mae',
                       'ty': 'mae'})

# Train the model
model.fit(X_train, {'nail_spacing': Y_train_onehot_nail_spacing, 
                    'num_sheathing_panels': Y_train_onehot_num_sheathing_panels,
                    'num_end_studs': Y_train_onehot_num_end_studs,
                    'total_num_studs': Y_train['Total number studs'],
                    'holddown_model': Y_train['HoldDown Model'],
                    'tx': Y_train['Tx(s)'],
                    'ty': Y_train['Ty(s)']},
          epochs=100, batch_size=32, validation_split=0.2)



Epoch 1/100
1246/1246 [==============================] - 10s 5ms/step - loss: 96.6684 - nail_spacing_loss: 0.0991 - num_sheathing_panels_loss: 0.3223 - num_end_studs_loss: 1.0217 - total_num_studs_loss: 64.6923 - holddown_model_loss: 30.4776 - tx_loss: 0.0293 - ty_loss: 0.0261 - nail_spacing_accuracy: 0.9899 - num_sheathing_panels_accuracy: 0.8697 - num_end_studs_accuracy: 0.6442 - total_num_studs_mae: 6.3216 - holddown_model_mae: 3.4358 - tx_mae: 0.1115 - ty_mae: 0.1016 - val_loss: 47.8061 - val_nail_spacing_loss: 0.0081 - val_num_sheathing_panels_loss: 0.2715 - val_num_end_studs_loss: 0.7845 - val_total_num_studs_loss: 24.9474 - val_holddown_model_loss: 21.7844 - val_tx_loss: 0.0057 - val_ty_loss: 0.0046 - val_nail_spacing_accuracy: 1.0000 - val_num_sheathing_panels_accuracy: 0.8893 - val_num_end_studs_accuracy: 0.7330 - val_total_num_studs_mae: 3.4801 - val_holddown_model_mae: 3.1597 - val_tx_mae: 0.0598 - val_ty_mae: 0.0532
Epoch 2/100
1246/1246 [==============================] - 6

In [42]:
# Evaluate the model on the test set
evaluation_results = model.evaluate(X_test, {'nail_spacing': Y_test_onehot_nail_spacing, 
                                             'num_sheathing_panels': Y_test_onehot_num_sheathing_panels,
                                             'num_end_studs': Y_test_onehot_num_end_studs,
                                             'total_num_studs': Y_test['Total number studs'],
                                             'holddown_model': Y_test['HoldDown Model'],
                                             'tx': Y_test['Tx(s)'],
                                             'ty': Y_test['Ty(s)']},
                                    batch_size=32)

# Extract and print the accuracy for each output
output_metrics = {name: value for name, value in zip(model.metrics_names, evaluation_results)}
print("Metrics on Test Set:")
for output_name, metric_value in output_metrics.items():
    print(f"{output_name}: {metric_value}")


390/390 [==============================] - 1s 3ms/step - loss: 15.5008 - nail_spacing_loss: 0.0000e+00 - num_sheathing_panels_loss: 0.2443 - num_end_studs_loss: 0.3613 - total_num_studs_loss: 2.0881 - holddown_model_loss: 12.7973 - tx_loss: 0.0055 - ty_loss: 0.0043 - nail_spacing_accuracy: 1.0000 - num_sheathing_panels_accuracy: 0.8929 - num_end_studs_accuracy: 0.8455 - total_num_studs_mae: 0.9306 - holddown_model_mae: 2.6309 - tx_mae: 0.0583 - ty_mae: 0.0514
Metrics on Test Set:
loss: 15.500787734985352
nail_spacing_loss: 0.0
num_sheathing_panels_loss: 0.2442822903394699
num_end_studs_loss: 0.36130228638648987
total_num_studs_loss: 2.088104009628296
holddown_model_loss: 12.797307968139648
tx_loss: 0.005488893017172813
ty_loss: 0.004300387110561132
nail_spacing_accuracy: 1.0
num_sheathing_panels_accuracy: 0.8928772211074829
num_end_studs_accuracy: 0.8454990983009338
total_num_studs_mae: 0.9306480884552002
holddown_model_mae: 2.630870819091797
tx_mae: 0.058348290622234344
ty_mae: 0.0514

### 2. 
- Try different regression models to improve regression loss

#### Predicting Values of C

In [6]:
path = './Files/After_Feature_Engineering/Split/'

X_train_C_1 = pd.read_csv(path + "X_train_C_part1_FE.csv", low_memory=False)
X_test_C_1 = pd.read_csv(path + "X_test_C_part1_FE.csv", low_memory=False)

Y_train_C_1 = pd.read_csv(path + "Y_train_C_part1_FE.csv", low_memory=False)
Y_test_C_1 = pd.read_csv(path + "Y_test_C_part1_FE.csv", low_memory=False)

In [6]:
# Define the input layer
input_layer_C_1 = Input(shape=(X_train_C_1.shape[1],))

# One hot encoding Y
Y_train_onehot_nail_spacing, Y_test_onehot_nail_spacing, Y_train_onehot_num_sheathing_panels, Y_test_onehot_num_sheathing_panels, Y_train_onehot_num_end_studs, Y_test_onehot_num_end_studs, Y_train_C_1, Y_test_C_1 = label_encoder(Y_train_C_1, Y_test_C_1)


nail_spacing_output = Dense(3, activation='softmax', name='nail_spacing')(input_layer_C_1)
num_sheathing_panels_output = Dense(2, activation='softmax', name='num_sheathing_panels')(input_layer_C_1)
num_end_studs_output = Dense(6, activation='softmax', name='num_end_studs')(input_layer_C_1)
total_num_studs_output = Dense(1, activation='linear', name='total_num_studs')(input_layer_C_1)
holddown_model_output = Dense(1, activation='linear', name='holddown/ats')(input_layer_C_1)

# Combine the outputs into a single model
model_C_1 = Model(inputs=input_layer_C_1, outputs=[nail_spacing_output, num_sheathing_panels_output, 
                                           num_end_studs_output, total_num_studs_output, 
                                           holddown_model_output])

# Compile the model
model_C_1.compile(optimizer='adam', 
              loss={'nail_spacing': 'categorical_crossentropy',
                    'num_sheathing_panels': 'categorical_crossentropy',
                    'num_end_studs': 'categorical_crossentropy',
                    'total_num_studs': 'mean_squared_error',
                    'holddown/ats': 'mean_squared_error'
                    },
              metrics={'nail_spacing': 'accuracy',
                       'num_sheathing_panels': 'accuracy',
                       'num_end_studs': 'accuracy',
                       'total_num_studs': 'mae',
                       'holddown/ats': 'mae'
                       })

# Implement ReduceLROnPlateau callback
# Went from 0.02 to 0.008 in total loss with this line. Could be improved with cross-validation
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=10, min_lr=0.00001)


# Train the model
model_C_1.fit(X_train_C_1, {'nail_spacing': Y_train_onehot_nail_spacing, 
                    'num_sheathing_panels': Y_train_onehot_num_sheathing_panels,
                    'num_end_studs': Y_train_onehot_num_end_studs,
                    'total_num_studs': Y_train_C_1['Total number studs'],
                    'holddown/ats': Y_train_C_1['HoldDown Model / ATS']
                    },
          epochs=100, batch_size=32, validation_data = (X_test_C_1, {'nail_spacing': Y_test_onehot_nail_spacing,
                                                                        'num_sheathing_panels': Y_test_onehot_num_sheathing_panels,
                                                                        'num_end_studs': Y_test_onehot_num_end_studs,
                                                                        'total_num_studs': Y_test_C_1['Total number studs'],
                                                                        'holddown/ats': Y_test_C_1['HoldDown Model / ATS']
                                             }), callbacks=[reduce_lr])

Epoch 1/100
1557/1557 [==============================] - 9s 4ms/step - loss: 90.2302 - nail_spacing_loss: 0.7171 - num_sheathing_panels_loss: 0.3144 - num_end_studs_loss: 0.9525 - total_num_studs_loss: 58.1934 - holddown/ats_loss: 30.0529 - nail_spacing_accuracy: 0.7029 - num_sheathing_panels_accuracy: 0.8749 - num_end_studs_accuracy: 0.6723 - total_num_studs_mae: 5.8893 - holddown/ats_mae: 3.4853 - val_loss: 44.9894 - val_nail_spacing_loss: 0.6375 - val_num_sheathing_panels_loss: 0.2691 - val_num_end_studs_loss: 0.7402 - val_total_num_studs_loss: 21.2035 - val_holddown/ats_loss: 22.1392 - val_nail_spacing_accuracy: 0.7401 - val_num_sheathing_panels_accuracy: 0.8914 - val_num_end_studs_accuracy: 0.7562 - val_total_num_studs_mae: 3.2431 - val_holddown/ats_mae: 3.1741 - lr: 0.0010
Epoch 2/100
1557/1557 [==============================] - 8s 5ms/step - loss: 36.2964 - nail_spacing_loss: 0.6110 - num_sheathing_panels_loss: 0.2569 - num_end_studs_loss: 0.6778 - total_num_studs_loss: 16.3196 

KeyboardInterrupt: 

In [12]:
# Evaluate the model on the test set
evaluation_results = model_C_1.evaluate(X_test_C_1, {'nail_spacing': Y_test_onehot_nail_spacing, 
                                             'num_sheathing_panels': Y_test_onehot_num_sheathing_panels,
                                             'num_end_studs': Y_test_onehot_num_end_studs,
                                             'total_num_studs': Y_test_C_1['Total number studs'],
                                             'holddown/ats': Y_test_C_1['HoldDown Model / ATS']
                                             },
                                    batch_size=32)

# Extract and print the accuracy for each output
output_metrics = {name: value for name, value in zip(model_C_1.metrics_names, evaluation_results)}
print("Metrics on Test Set:")
for output_name, metric_value in output_metrics.items():
    print(f"{output_name}: {metric_value}")

390/390 [==============================] - 1s 3ms/step - loss: 16.0792 - nail_spacing_loss: 0.5877 - num_sheathing_panels_loss: 0.2440 - num_end_studs_loss: 0.3573 - total_num_studs_loss: 2.0877 - holddown/ats_loss: 12.8025 - nail_spacing_accuracy: 0.7528 - num_sheathing_panels_accuracy: 0.8918 - num_end_studs_accuracy: 0.8500 - total_num_studs_mae: 0.9390 - holddown/ats_mae: 2.6588
Metrics on Test Set:
loss: 16.07921028137207
nail_spacing_loss: 0.5876692533493042
num_sheathing_panels_loss: 0.24397161602973938
num_end_studs_loss: 0.3573315739631653
total_num_studs_loss: 2.0877490043640137
holddown/ats_loss: 12.802483558654785
nail_spacing_accuracy: 0.7528306245803833
num_sheathing_panels_accuracy: 0.8918333053588867
num_end_studs_accuracy: 0.8499959707260132
total_num_studs_mae: 0.9390494227409363
holddown/ats_mae: 2.6588473320007324


#### Predicting values of Tx and Ty

In [7]:
X_train_C_2 = pd.read_csv(path + "X_train_C_part2_FE.csv", low_memory=False)
X_test_C_2 = pd.read_csv(path + "X_test_C_part2_FE.csv", low_memory=False)

Y_train_C_2 = pd.read_csv(path + "Y_train_C_part2_FE.csv", low_memory=False)
Y_test_C_2 = pd.read_csv(path + "Y_test_C_part2_FE.csv", low_memory=False)

In [8]:

# Define the input layer
input_layer_C_2 = Input(shape=(X_train_C_2.shape[1],))

# Define separate branches for each output
# for linear we have mae: 0.056 and 0.049
# for relu we have mae: 0.056 and 0.047
# for leaky_relu we have mae: 0.056 and 0.048
# for swish we have mae: 0.054 and 0.046
# for softplus, we have mae: 0.055 and 0.046
# for tanh, we have mae: 0.064 and 0.052
tx_output = Dense(1, activation='swish', name='tx')(input_layer_C_2)
ty_output = Dense(1, activation='swish', name='ty')(input_layer_C_2)

# Combine the outputs into a single model
model_C_2 = Model(inputs=input_layer_C_2, outputs=[tx_output, ty_output])

# Adjusted optimizer with a lower learning rate
#optimizer = keras.optimizers.Adam(learning_rate=0.0001) # use cross-validation to find the optimal learning rate

# Compile the model
model_C_2.compile(optimizer='adam', 
              loss={
                    'tx': 'mean_squared_error',
                    'ty': 'mean_squared_error'
                    },
              metrics={
                       'tx': 'mae',
                       'ty': 'mae'
                       })

# Implement ReduceLROnPlateau callback
# Went from 0.02 to 0.008 in total loss with this line. Could be improved with cross-validation
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=10, min_lr=0.00001)


# Train the model
model_C_2.fit(X_train_C_2, {
                        'tx': Y_train_C_2['Tx(s)'],
                        'ty': Y_train_C_2['Ty(s)']
                        },
          epochs=100, batch_size=32, validation_data = (X_test_C_2, {
                                                    'tx': Y_test_C_2['Tx(s)'],
                                                    'ty': Y_test_C_2['Ty(s)']
                                             }), callbacks=[reduce_lr])


Epoch 1/100
5/5 [==============================] - 2s 204ms/step - loss: 0.6463 - tx_loss: 0.3580 - ty_loss: 0.2883 - tx_mae: 0.4994 - ty_mae: 0.4632 - val_loss: 0.2839 - val_tx_loss: 0.1373 - val_ty_loss: 0.1466 - val_tx_mae: 0.2953 - val_ty_mae: 0.3471 - lr: 0.0010
Epoch 2/100
5/5 [==============================] - 0s 14ms/step - loss: 0.1763 - tx_loss: 0.1071 - ty_loss: 0.0692 - tx_mae: 0.2595 - ty_mae: 0.2131 - val_loss: 0.1746 - val_tx_loss: 0.0941 - val_ty_loss: 0.0805 - val_tx_mae: 0.2395 - val_ty_mae: 0.2304 - lr: 0.0010
Epoch 3/100
5/5 [==============================] - 0s 17ms/step - loss: 0.1331 - tx_loss: 0.0783 - ty_loss: 0.0548 - tx_mae: 0.2209 - ty_mae: 0.1935 - val_loss: 0.1532 - val_tx_loss: 0.0830 - val_ty_loss: 0.0701 - val_tx_mae: 0.2457 - val_ty_mae: 0.2284 - lr: 0.0010
Epoch 4/100
5/5 [==============================] - 0s 24ms/step - loss: 0.1137 - tx_loss: 0.0621 - ty_loss: 0.0516 - tx_mae: 0.2019 - ty_mae: 0.1827 - val_loss: 0.0820 - val_tx_loss: 0.0547 - val_ty

In [14]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

# Number of models to create
n_estimators = 5

# List to store individual model predictions
pred_tx_list = []
pred_ty_list = []

for _ in range(n_estimators):
    tx_output = Dense(1, activation='swish', name='tx')(input_layer_C_2)
    ty_output = Dense(1, activation='swish', name='ty')(input_layer_C_2)

    # Combine the outputs into a single model
    model = Model(inputs=input_layer_C_2, outputs=[tx_output, ty_output])

    # Compile the model
    model.compile(optimizer='adam', 
                loss={
                        'tx': 'mean_squared_error',
                        'ty': 'mean_squared_error'
                        },
                metrics={
                        'tx': 'mae',
                        'ty': 'mae'
                        })
    
    # Train the model on the training data
    model.fit(X_train_C_2, {'tx': Y_train_C_2['Tx(s)'], 'ty': Y_train_C_2['Ty(s)']}, epochs=100, batch_size=32)

    # Make predictions on the test set
    predictions = model.predict(X_test_C_2)

    # Assuming the output shape is (batch_size, 1) for each output
    pred_tx = predictions[0][:, 0]
    pred_ty = predictions[1][:, 0]
    
    # Store predictions
    pred_tx_list.append(pred_tx)
    pred_ty_list.append(pred_ty)

# Combine predictions from all models
final_pred_tx = np.mean(pred_tx_list, axis=0)
final_pred_ty = np.mean(pred_ty_list, axis=0)

# Evaluate the performance
mse_tx = mean_squared_error(Y_test_C_2['Tx(s)'], final_pred_tx)
mae_tx = mean_absolute_error(Y_test_C_2['Tx(s)'], final_pred_tx)

mse_ty = mean_squared_error(Y_test_C_2['Ty(s)'], final_pred_ty)
mae_ty = mean_absolute_error(Y_test_C_2['Ty(s)'], final_pred_ty)

print(f'Tx MSE: {mse_tx}, Tx MAE: {mae_tx}')
print(f'Ty MSE: {mse_ty}, Ty MAE: {mae_ty}')



Epoch 1/100
5/5 [==============================] - 1s 3ms/step - loss: 0.3829 - tx_loss: 0.2646 - ty_loss: 0.1183 - tx_mae: 0.4288 - ty_mae: 0.2881
Epoch 2/100
5/5 [==============================] - 0s 6ms/step - loss: 0.1967 - tx_loss: 0.1152 - ty_loss: 0.0815 - tx_mae: 0.2777 - ty_mae: 0.2295
Epoch 3/100
5/5 [==============================] - 0s 3ms/step - loss: 0.1422 - tx_loss: 0.0822 - ty_loss: 0.0600 - tx_mae: 0.2389 - ty_mae: 0.2046
Epoch 4/100
5/5 [==============================] - 0s 2ms/step - loss: 0.1095 - tx_loss: 0.0652 - ty_loss: 0.0444 - tx_mae: 0.2041 - ty_mae: 0.1663
Epoch 5/100
5/5 [==============================] - 0s 16ms/step - loss: 0.0728 - tx_loss: 0.0436 - ty_loss: 0.0291 - tx_mae: 0.1755 - ty_mae: 0.1419
Epoch 6/100
5/5 [==============================] - 0s 4ms/step - loss: 0.0526 - tx_loss: 0.0279 - ty_loss: 0.0247 - tx_mae: 0.1336 - ty_mae: 0.1285
Epoch 7/100
5/5 [==============================] - 0s 4ms/step - loss: 0.0471 - tx_loss: 0.0273 - ty_loss: 0.01

In [5]:
def create_model_C_2(learning_rate=0.001, activation='swish', factor=0.2, patience=10, min_lr=0.00001):
    input_layer_C_2 = Input(shape=(X_train_C_2.shape[1],))
    tx_output = Dense(1, activation=activation, name='tx')(input_layer_C_2)
    ty_output = Dense(1, activation=activation, name='ty')(input_layer_C_2)

    model = Model(inputs=input_layer_C_2, outputs=[tx_output, ty_output])

    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer,
                  loss={'tx': 'mean_squared_error', 'ty': 'mean_squared_error'},
                  metrics={'tx': 'mae', 'ty': 'mae'})

    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=factor, patience=patience, min_lr=min_lr)
    model.callbacks = [reduce_lr]
    return model

In [21]:
from sklearn.model_selection import KFold
# Define the parameter grid for GridSearchCV
param_grid = {
    'model__learning_rate': [0.001, 0.0001],
    'model__activation': ['linear', 'relu', 'leaky_relu', 'swish', 'softplus', 'tanh'],
    'epochs': [50, 100, 150],
    'model__batch_size': [16, 32, 64],
    'model__factor': [0.1, 0.2],
    'model__patience': [5, 10, 15],
    'model__min_lr': [0.00001, 0.0001]
}


# Modify your create_model_C_2 function to accept a 'model_type' argument
def create_model_C_2(model_type='tx', learning_rate=0.001, activation='swish', factor=0.2, patience=10, min_lr=0.00001, **kwargs):
    input_layer_C_2 = Input(shape=(X_train_C_2.shape[1],))
    
    # Use the 'model_type' argument to determine which output to create
    if model_type == 'tx':
        output = Dense(1, activation=activation, name='tx')(input_layer_C_2)
    elif model_type == 'ty':
        output = Dense(1, activation=activation, name='ty')(input_layer_C_2)
    else:
        raise ValueError("Invalid model_type. Use 'tx' or 'ty'.")

    model = Model(inputs=input_layer_C_2, outputs=output)

    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer,
                  loss='mean_squared_error',
                  metrics='mae')

    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=factor, patience=patience, min_lr=min_lr)
    model.callbacks = [reduce_lr]
    return model


# Wrap the Keras model using KerasRegressor
keras_regressor_tx = KerasRegressor(model=lambda learning_rate, activation, factor, patience, min_lr, batch_size:
    create_model_C_2(model_type='tx', learning_rate=learning_rate, activation=activation, factor=factor, patience=patience, min_lr=min_lr, batch_size=batch_size),
    epochs=50, verbose=0)

keras_regressor_ty = KerasRegressor(model=lambda learning_rate, activation, factor, patience, min_lr, batch_size:
    create_model_C_2(model_type='ty', learning_rate=learning_rate, activation=activation, factor=factor, patience=patience, min_lr=min_lr, batch_size=batch_size),
    epochs=50, verbose=0)


# Use KFold cross-validation
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# Create the GridSearchCV instance for both 'tx' and 'ty'
grid_search_tx = GridSearchCV(estimator=keras_regressor_tx, param_grid=param_grid, cv=cv, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search_ty = GridSearchCV(estimator=keras_regressor_ty, param_grid=param_grid, cv=cv, scoring='neg_mean_squared_error', n_jobs=-1)

# Fit the GridSearchCV instance to your data
grid_search_result_tx = grid_search_tx.fit(X_train_C_2, Y_train_C_2['Tx(s)'])  # Use 'tx' column of Y_train_C_2
grid_search_result_ty = grid_search_ty.fit(X_train_C_2, Y_train_C_2['Ty(s)'])  # Use 'ty' column of Y_train_C_2

# Print the best parameters and their corresponding mean test scores
print("Best Parameters (tx): ", grid_search_result_tx.best_params_)
print("Best Mean Test Score (tx): ", grid_search_result_tx.best_score_)

print("Best Parameters (ty): ", grid_search_result_ty.best_params_)
print("Best Mean Test Score (ty): ", grid_search_result_ty.best_score_)




c:\Users\anejj\anaconda3\lib\site-packages\joblib\externals\loky\process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
c:\Users\anejj\anaconda3\lib\site-packages\joblib\externals\loky\process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Best Parameters (tx):  {'epochs': 150, 'model__activation': 'softplus', 'model__batch_size': 32, 'model__factor': 0.1, 'model__learning_rate': 0.001, 'model__min_lr': 1e-05, 'model__patience': 15}
Best Mean Test Score (tx):  -0.0061000747557187745
Best Parameters (ty):  {'epochs': 150, 'model__activation': 'softplus', 'model__batch_size': 32, 'model__factor': 0.1, 'model__learning_rate': 0.001, 'model__min_lr': 0.0001, 'model__patience': 15}
Best Mean Test Score (ty):  -0.004111967720104843


In [35]:
# Evaluate the model on the test set
evaluation_results = model_C_2.evaluate(X_test_C_2, {
                                                    'tx': Y_test_C_2['Tx(s)'],
                                                    'ty': Y_test_C_2['Ty(s)']
                                             },
                                    batch_size=32)

# Extract and print the accuracy for each output
output_metrics = {name: value for name, value in zip(model_C_2.metrics_names, evaluation_results)}
print("Metrics on Test Set:")
for output_name, metric_value in output_metrics.items():
    print(f"{output_name}: {metric_value}")

2/2 [==============================] - 0s 0s/step - loss: 0.0079 - tx_loss: 0.0045 - ty_loss: 0.0034 - tx_mae: 0.0554 - ty_mae: 0.0465
Metrics on Test Set:
loss: 0.007910527288913727
tx_loss: 0.004539818037301302
ty_loss: 0.003370709018781781
tx_mae: 0.055375732481479645
ty_mae: 0.046547435224056244


#### Predicting values of D 

In [52]:
X_train_D = df = pd.read_csv(path + "X_train_D_FE.csv", low_memory=False)
X_test_D = pd.read_csv(path + "X_test_D_FE.csv", low_memory=False)

Y_train_D = df = pd.read_csv(path + "Y_train_d_FE.csv", low_memory=False)
Y_test_D = pd.read_csv(path + "Y_test_D_FE.csv", low_memory=False)

# Chnage the column name
column_names = ['omega_x', 'omega_y', 'mu_x', 'mu_y', 'cmr', 'ssf','acmr', 'io-ln', 'io-b', 'ls-ln', 'ls-b', 'cp-ln', 'cp-b']
Y_train_D.columns = column_names
Y_test_D.columns = column_names

display(Y_train_D)

,omega_x,omega_y,mu_x,mu_y,cmr,ssf,acmr,io-ln,io-b,ls-ln,ls-b,cp-ln,cp-b
0,3.19,3.62,4.61,5.65,1.07,1.29,1.65,-0.785,0.222,-0.224,0.259,0.108,0.274
1,5.67,5.43,4.27,4.28,2.85,1.26,4.31,-1.433,0.401,-0.838,0.396,-0.525,0.389
2,5.55,5.88,4.60,4.62,2.61,1.21,3.77,-1.111,0.314,-0.548,0.346,-0.255,0.378
3,4.54,3.18,3.32,5.49,1.87,1.12,2.52,-0.396,0.223,0.109,0.276,0.419,0.344
4,3.26,4.11,5.45,6.00,1.78,1.31,2.79,-0.096,0.344,0.445,0.372,0.742,0.420
...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,4.68,5.81,3.35,4.43,2.24,1.12,3.00,-0.364,0.248,0.262,0.276,0.641,0.300
156,4.35,4.36,4.54,4.35,1.56,1.18,2.21,-0.751,0.268,-0.107,0.285,0.260,0.305
157,4.09,4.92,5.30,5.20,2.18,1.42,3.70,-1.061,0.312,-0.493,0.327,-0.176,0.364
158,6.02,5.41,5.33,5.20,3.30,1.14,4.53,-0.573,0.236,-0.009,0.276,0.322,0.318


In [53]:
# Define the input layer
input_layer_D = Input(shape=(X_train_D.shape[1],))

# Define separate branches for each output
omega_x_output = Dense(1, activation='linear', name='omega_x')(input_layer_D)
omega_y_output = Dense(1, activation='linear', name='omega_y')(input_layer_D)
mu_x_output = Dense(1, activation='linear', name='mu_x')(input_layer_D)
mu_y_output = Dense(1, activation='linear', name='mu_y')(input_layer_D)
cmr_output = Dense(1, activation='linear', name='cmr')(input_layer_D)
ssf_output = Dense(1, activation='linear', name='ssf')(input_layer_D)
acmr_output = Dense(1, activation='linear', name='acmr')(input_layer_D)
io_ln_output = Dense(1, activation='linear', name='io-ln')(input_layer_D)
io_b_output = Dense(1, activation='linear', name='io-b')(input_layer_D)
ls_ln_output = Dense(1, activation='linear', name='ls-ln')(input_layer_D)
ls_b_output = Dense(1, activation='linear', name='ls-b')(input_layer_D)
cp_ln_output = Dense(1, activation='linear', name='cp-ln')(input_layer_D)
cp_b_output = Dense(1, activation='linear', name='cp-b')(input_layer_D)

# Combine the outputs into a single model
model_D = Model(inputs=input_layer_D, outputs=[omega_x_output, omega_y_output, 
                                               mu_x_output, mu_y_output,
                                               cmr_output, ssf_output, acmr_output,
                                               io_ln_output, io_b_output,
                                               ls_ln_output, ls_b_output,
                                               cp_ln_output, cp_b_output])

# Compile the model
model_D.compile(optimizer='adam', 
              loss={
                        'omega_x': 'mean_squared_error',
                        'omega_y': 'mean_squared_error',
                        'mu_x': 'mean_squared_error',
                        'mu_y': 'mean_squared_error',
                        'cmr': 'mean_squared_error',
                        'ssf': 'mean_squared_error',
                        'acmr': 'mean_squared_error',
                        'io-ln': 'mean_squared_error',
                        'io-b': 'mean_squared_error',
                        'ls-ln': 'mean_squared_error',
                        'ls-b': 'mean_squared_error',
                        'cp-ln': 'mean_squared_error',
                        'cp-b': 'mean_squared_error'
                    },
              metrics={
                       'omega_x': 'mae',
                        'omega_y': 'mae',
                        'mu_x': 'mae',
                        'mu_y': 'mae',
                        'cmr': 'mae',
                        'ssf': 'mae',
                        'acmr': 'mae',
                        'io-ln': 'mae',
                        'io-b': 'mae',
                        'ls-ln': 'mae',
                        'ls-b': 'mae',
                        'cp-ln': 'mae',
                        'cp-b': 'mae'
                       })
# Implement ReduceLROnPlateau callback
# Went from 6.8 to 6.9 in total loss with this line. Could be improved with cross-validation
#reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=10, min_lr=0.00001)

# Train the model
model_D.fit(X_train_D, {
                        'omega_x': Y_train_D['omega_x'],
                        'omega_y': Y_train_D['omega_y'],
                        'mu_x': Y_train_D['mu_x'],
                        'mu_y': Y_train_D['mu_y'],
                        'cmr': Y_train_D['cmr'],
                        'ssf': Y_train_D['ssf'],
                        'acmr': Y_train_D['acmr'],
                        'io-ln': Y_train_D['io-ln'],
                        'io-b': Y_train_D['io-b'],
                        'ls-ln': Y_train_D['ls-ln'],
                        'ls-b': Y_train_D['ls-b'],
                        'cp-ln': Y_train_D['cp-ln'],
                        'cp-b': Y_train_D['cp-b']
                        
                        },
          epochs=100, batch_size=32, validation_data=(X_test_D, {
                                                            'omega_x': Y_test_D['omega_x'],
                                                            'omega_y': Y_test_D['omega_y'],
                                                            'mu_x': Y_test_D['mu_x'],
                                                            'mu_y': Y_test_D['mu_y'],
                                                            'cmr': Y_test_D['cmr'],
                                                            'ssf': Y_test_D['ssf'],
                                                            'acmr': Y_test_D['acmr'],
                                                            'io-ln': Y_test_D['io-ln'],
                                                            'io-b': Y_test_D['io-b'],
                                                            'ls-ln': Y_test_D['ls-ln'],
                                                            'ls-b': Y_test_D['ls-b'],
                                                            'cp-ln': Y_test_D['cp-ln'],
                                                            'cp-b': Y_test_D['cp-b']
                                                            }
                                                  ))

Epoch 1/100
5/5 [==============================] - 6s 560ms/step - loss: 69.8757 - omega_x_loss: 16.8158 - omega_y_loss: 17.9923 - mu_x_loss: 5.9912 - mu_y_loss: 8.3179 - cmr_loss: 2.3618 - ssf_loss: 1.4306 - acmr_loss: 8.5084 - io-ln_loss: 1.3135 - io-b_loss: 1.1072 - ls-ln_loss: 1.2649 - ls-b_loss: 1.7993 - cp-ln_loss: 1.2026 - cp-b_loss: 1.7702 - omega_x_mae: 3.3329 - omega_y_mae: 3.4482 - mu_x_mae: 2.1211 - mu_y_mae: 2.3918 - cmr_mae: 1.3002 - ssf_mae: 1.0447 - acmr_mae: 2.3883 - io-ln_mae: 0.9793 - io-b_mae: 0.9449 - ls-ln_mae: 0.9900 - ls-b_mae: 1.1218 - cp-ln_mae: 0.9187 - cp-b_mae: 1.1231 - val_loss: 63.9679 - val_omega_x_loss: 17.6079 - val_omega_y_loss: 15.7938 - val_mu_x_loss: 5.1838 - val_mu_y_loss: 7.8062 - val_cmr_loss: 0.9932 - val_ssf_loss: 1.1078 - val_acmr_loss: 8.1336 - val_io-ln_loss: 0.8787 - val_io-b_loss: 0.8577 - val_ls-ln_loss: 1.0091 - val_ls-b_loss: 1.8764 - val_cp-ln_loss: 0.9967 - val_cp-b_loss: 1.7230 - val_omega_x_mae: 3.6067 - val_omega_y_mae: 3.5382 - v

In [ ]:
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras.callbacks import ReduceLROnPlateau

def create_model_D(learning_rate=0.001, activation='linear', factor=0.2, patience=10, min_lr=0.00001):
    # Define the input layer
    input_layer_D = Input(shape=(X_train_D.shape[1],))

    # Define separate branches for each output
    omega_x_output = Dense(1, activation='linear', name='omega_x')(input_layer_D)
    omega_y_output = Dense(1, activation='linear', name='omega_y')(input_layer_D)
    mu_x_output = Dense(1, activation='linear', name='mu_x')(input_layer_D)
    mu_y_output = Dense(1, activation='linear', name='mu_y')(input_layer_D)
    cmr_output = Dense(1, activation='linear', name='cmr')(input_layer_D)
    ssf_output = Dense(1, activation='linear', name='ssf')(input_layer_D)
    acmr_output = Dense(1, activation='linear', name='acmr')(input_layer_D)
    io_ln_output = Dense(1, activation='linear', name='io-ln')(input_layer_D)
    io_b_output = Dense(1, activation='linear', name='io-b')(input_layer_D)
    ls_ln_output = Dense(1, activation='linear', name='ls-ln')(input_layer_D)
    ls_b_output = Dense(1, activation='linear', name='ls-b')(input_layer_D)
    cp_ln_output = Dense(1, activation='linear', name='cp-ln')(input_layer_D)
    cp_b_output = Dense(1, activation='linear', name='cp-b')(input_layer_D)

    # Combine the outputs into a single model
    model_D = Model(inputs=input_layer_D, outputs=[omega_x_output, omega_y_output, 
                                                mu_x_output, mu_y_output,
                                                cmr_output, ssf_output, acmr_output,
                                                io_ln_output, io_b_output,
                                                ls_ln_output, ls_b_output,
                                                cp_ln_output, cp_b_output])

    # Compile the model
    model_D.compile(optimizer='adam', 
                    loss={
                        'omega_x': 'mean_squared_error',
                        'omega_y': 'mean_squared_error',
                        'mu_x': 'mean_squared_error',
                        'mu_y': 'mean_squared_error',
                        'cmr': 'mean_squared_error',
                        'ssf': 'mean_squared_error',
                        'acmr': 'mean_squared_error',
                        'io-ln': 'mean_squared_error',
                        'io-b': 'mean_squared_error',
                        'ls-ln': 'mean_squared_error',
                        'ls-b': 'mean_squared_error',
                        'cp-ln': 'mean_squared_error',
                        'cp-b': 'mean_squared_error'
                    },
                    metrics={
                        'omega_x': 'mae',
                        'omega_y': 'mae',
                        'mu_x': 'mae',
                        'mu_y': 'mae',
                        'cmr': 'mae',
                        'ssf': 'mae',
                        'acmr': 'mae',
                        'io-ln': 'mae',
                        'io-b': 'mae',
                        'ls-ln': 'mae',
                        'ls-b': 'mae',
                        'cp-ln': 'mae',
                        'cp-b': 'mae'
                    })

    # Implement ReduceLROnPlateau callback
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=10, min_lr=0.00001)
    model_D.callbacks = [reduce_lr]

    # Train the model
    history_D = model_D.fit(X_train_D, {
                                'omega_x': Y_train_D['omega_x'],
                                'omega_y': Y_train_D['omega_y'],
                                'mu_x': Y_train_D['mu_x'],
                                'mu_y': Y_train_D['mu_y'],
                                'cmr': Y_train_D['cmr'],
                                'ssf': Y_train_D['ssf'],
                                'acmr': Y_train_D['acmr'],
                                'io-ln': Y_train_D['io-ln'],
                                'io-b': Y_train_D['io-b'],
                                'ls-ln': Y_train_D['ls-ln'],
                                'ls-b': Y_train_D['ls-b'],
                                'cp-ln': Y_train_D['cp-ln'],
                                'cp-b': Y_train_D['cp-b']
                            },
                            epochs=100, batch_size=32,
                            validation_data=(X_test_D, {
                                'omega_x': Y_test_D['omega_x'],
                                'omega_y': Y_test_D['omega_y'],
                                'mu_x': Y_test_D['mu_x'],
                                'mu_y': Y_test_D['mu_y'],
                                'cmr': Y_test_D['cmr'],
                                'ssf': Y_test_D['ssf'],
                                'acmr': Y_test_D['acmr'],
                                'io-ln': Y_test_D['io-ln'],
                                'io-b': Y_test_D['io-b'],
                                'ls-ln': Y_test_D['ls-ln'],
                                'ls-b': Y_test_D['ls-b'],
                                'cp-ln': Y_test_D['cp-ln'],
                                'cp-b': Y_test_D['cp-b']
                            }))


    # Add ReduceLROnPlateau callback with the specified parameters
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=factor, patience=patience, min_lr=min_lr)
    model_D.callbacks = [reduce_lr]

    return model_D

# Define the parameter grid for GridSearchCV
param_grid_D = {
    'learning_rate': [0.001, 0.0001],
    'activation': ['linear', 'relu', 'leaky_relu', 'swish', 'softplus', 'tanh'],
    'epochs': [50, 100, 150],
    'batch_size': [16, 32, 64],
    'factor': [0.1, 0.2, 0.3],
    'patience': [5, 10, 15],
    'min_lr': [0.00001, 0.000001]
}

# Create a KerasRegressor for use with GridSearchCV
neural_network_model_D = KerasRegressor(build_fn=create_model_D, verbose=0)

# Create GridSearchCV instance
grid_search_D = GridSearchCV(neural_network_model_D, param_grid_D, cv=3, scoring='neg_mean_absolute_error', n_jobs=-1)

# Fit the model
grid_result_D = grid_search_D.fit(X_train_D, {
    'omega_x': Y_train_D['omega_x'],
    'omega_y': Y_train_D['omega_y'],
    'mu_x': Y_train_D['mu_x'],
    'mu_y': Y_train_D['mu_y'],
    'cmr': Y_train_D['cmr'],
    'ssf': Y_train_D['ssf'],
    'acmr': Y_train_D['acmr'],
    'io-ln': Y_train_D['io-ln'],
    'io-b': Y_train_D['io-b'],
    'ls-ln': Y_train_D['ls-ln'],
    'ls-b': Y_train_D['ls-b'],
    'cp-ln': Y_train_D['cp-ln'],
    'cp-b': Y_train_D['cp-b']
})

# Print the best parameters and their corresponding mean test scores
print("Best Parameters: ", grid_result_D.best_params_)
print("Best Mean Test Score: ", grid_result_D.best_score_)


In [54]:
# Evaluate the model on the test set
evaluation_results = model_D.evaluate(X_test_D, {
                                                    'omega_x': Y_test_D['omega_x'],
                                                    'omega_y': Y_test_D['omega_y'],
                                                    'mu_x': Y_test_D['mu_x'],
                                                    'mu_y': Y_test_D['mu_y'],
                                                    'cmr': Y_test_D['cmr'],
                                                    'ssf': Y_test_D['ssf'],
                                                    'acmr': Y_test_D['acmr'],
                                                    'io-ln': Y_test_D['io-ln'],
                                                    'io-b': Y_test_D['io-b'],
                                                    'ls-ln': Y_test_D['ls-ln'],
                                                    'ls-b': Y_test_D['ls-b'],
                                                    'cp-ln': Y_test_D['cp-ln'],
                                                    'cp-b': Y_test_D['cp-b']
                                             },
                                    batch_size=32)

# Extract and print the accuracy for each output
output_metrics = {name: value for name, value in zip(model_D.metrics_names, evaluation_results)}
print("Metrics on Test Set:")
for output_name, metric_value in output_metrics.items():
    print(f"{output_name}: {metric_value}")

2/2 [==============================] - 0s 4ms/step - loss: 8.6226 - omega_x_loss: 3.0946 - omega_y_loss: 3.4811 - mu_x_loss: 0.1671 - mu_y_loss: 0.1972 - cmr_loss: 0.5798 - ssf_loss: 0.0047 - acmr_loss: 1.0242 - io-ln_loss: 0.0110 - io-b_loss: 0.0074 - ls-ln_loss: 0.0201 - ls-b_loss: 0.0097 - cp-ln_loss: 0.0187 - cp-b_loss: 0.0070 - omega_x_mae: 1.2126 - omega_y_mae: 1.4386 - mu_x_mae: 0.2894 - mu_y_mae: 0.2503 - cmr_mae: 0.5872 - ssf_mae: 0.0520 - acmr_mae: 0.8230 - io-ln_mae: 0.0790 - io-b_mae: 0.0646 - ls-ln_mae: 0.1013 - ls-b_mae: 0.0663 - cp-ln_mae: 0.1084 - cp-b_mae: 0.0645
Metrics on Test Set:
loss: 8.622621536254883
omega_x_loss: 3.094630718231201
omega_y_loss: 3.4810569286346436
mu_x_loss: 0.16709066927433014
mu_y_loss: 0.1972128301858902
cmr_loss: 0.5798074007034302
ssf_loss: 0.0046827709302306175
acmr_loss: 1.024198293685913
io-ln_loss: 0.011015506461262703
io-b_loss: 0.00742847565561533
ls-ln_loss: 0.020104745402932167
ls-b_loss: 0.009650805965065956
cp-ln_loss: 0.018741745